In [ ]:
!pip install -U spacy
!python -m spacy download fr_core_news_md
!pip install pdfplumber


In [ ]:
from google.colab import files
import pdfplumber

# window for uploading
uploaded = files.upload()

# getting the name of the uploaded dossier
pdf_path = list(uploaded.keys())[0]

# extracting text
text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"

# first 100 char of the text
print(text[:1000])


In [ ]:
keyword = input("لطفاً کلمه کلیدی (به زبان فرانسوی) را وارد کنید: ")


In [ ]:
import nltk
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')

stop_words_fr = set(stopwords.words('french'))


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

nltk.download('punkt')
nltk.download('stopwords')

stop_words_fr = set(stopwords.words('french'))

def get_context_words(text, keyword, window=2):
    tokens = word_tokenize(text, language='french')
    contexts = []
    for i, token in enumerate(tokens):
        if token.lower() == keyword.lower():
            start = max(0, i - window)
            end = min(len(tokens), i + window + 1)
            context = tokens[start:i] + tokens[i+1:end]
            context_filtered = [
                w.lower() for w in context
                if w.lower() not in stop_words_fr
                and w.lower() not in string.punctuation
                and w.isalpha()
            ]
            contexts.append(context_filtered)
    return contexts


In [ ]:
from nltk.corpus import stopwords

# std french stop words
nltk.download('stopwords')
stop_words_fr = set(stopwords.words('french'))

# mnl stop words
custom_stopwords = {
    'et', 'ou', 'le', 'la', 'un', 'une', 'du', 'de', 'des', 'à',
    ',', '.', ';', ':', '!', '?', 'l’', 'd’', '–', '-', '«', '»',
    'dans', 'en', 'pour', 'avec', 'par', 'sur', 'au', 'aux', 'ce', 'cet',
    'cette' , "ainsi" , "," , "plus", "tout", "comme", "a", "aux", "ainsi", "alors", "après", "aucun", "aucune",
    "autre", "avant", "avec", "avoir", "ça", "car", "ce", "cela", "cette", "cet", "ces",
    "cette", "dans", "donc", "dont", "du", "elle", "en", "est", "et", "eux", "il", "ils",
    "je", "là", "la", "le", "les", "leur", "lui", "ma", "mais", "me", "même", "mes",
     "moi", "mon", "ne", "nos", "notre", "nous", "on", "ou", "où", "par", "pas", "pour",
      "quand", "que", "qui", "sa", "se", "ses", "si", "son", "sont", "sur", "ta", "te",
       "tes", "toi", "ton", "tous", "tout", "trop", "tu", "un", "une", "vos", "votre",
      "vous", "y" , "of" , "the" , "in" , "for" , "b" , "and" , "vol" , "e" , "r" , "f" ,
        "v" , "ser" , "loi" , "re" , "sie" , "don" , "mots" , "titre" ,  "article", "code",
        "art", "alinéa", "mots", "remplace", "etat", "État",
       "rédigé", "disposition", "titre", "ar", "décret", "ministre", "ii", "l"
        "modifié", "président", "mentionnés", "présent", "articles", "cas",
         "modifie", "présente", "général", "représentant", "texte" , "ticle"
         , "genre" , "Conditions" , "Tacle" , "Charge" , "parc" , "." , ";" , ":" , "»"
}

stop_words_fr |= custom_stopwords


In [ ]:
print("⚠️ چند stopword فرانسوی:")
print(list(stop_words_fr)[:20])


In [ ]:
from collections import Counter

# just keep the alph/non-fr stop words
flat_words = [
    w.lower() for ctx in contexts for w in ctx
    if w.lower() not in stop_words_fr and w.isalpha()
]

most_common = Counter(flat_words).most_common(10)

print("📊 ۱۰ کلمه پرتکرار اطراف کلمه کلیدی (فیلترشده):")
for word, count in most_common:
    print(f"{word}: {count} بار")


In [ ]:
from transformers import CamembertModel, CamembertTokenizer
import torch

# Frnch model and tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")


In [ ]:
'''def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # استفاده از [CLS] توکن برای نمایه‌سازی


In [ ]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # using CLS


In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words_fr = set(stopwords.words('french'))


In [ ]:
'''from sklearn.metrics.pairwise import cosine_similarity

def analyze_similarity(contexts):
    sentences = [" ".join(ctx) for ctx in contexts]
    embeddings = [get_embedding(sen) for sen in sentences]

    # میانگین embedding کل زمینه‌ها
    avg_embedding = torch.stack(embeddings).mean(dim=0)

    seen_words = set()
    similarities = []

    for context in contexts:
        for word in context:
            if word in seen_words:
                continue
            seen_words.add(word)
            try:
                emb = get_embedding(word)
                sim = cosine_similarity(emb, avg_embedding).item()
                similarities.append((word, sim))
            except:
                continue  # اگر embedding نگرفت، ادامه بده

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:10]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import string

def analyze_similarity(contexts):
    sentences = [" ".join(ctx) for ctx in contexts]
    embeddings = [get_embedding(sen) for sen in sentences]

    avg_embedding = torch.stack(embeddings).mean(dim=0)

    similarities = []
    seen_words = set()

    for context in contexts:
        for word in context:
            word_clean = word.lower()

            #filter" stop word, pubctuation, numbers, already seen words
            # فیلتر: استاپ‌ورد، علامت‌نگارشی، عدد، و تکراری
            if (word_clean in stop_words_fr or
                word_clean in string.punctuation or
                not word_clean.isalpha() or
                word_clean in seen_words):
                continue

            seen_words.add(word_clean)

            try:
                emb = get_embedding(word_clean)
                sim = cosine_similarity(emb, avg_embedding).item()
                similarities.append((word_clean, sim))
            except:
                continue

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:10]


In [ ]:
semantic_top = analyze_similarity(contexts)

print("\n💡 ۱۰ کلمه با بیشترین شباهت معنایی به زمینه‌ی اطراف «{}»:".format(keyword))
for word, score in semantic_top:
    print(f"{word}: {score:.4f}")


In [ ]:
'''semantic_top = analyze_similarity(contexts)

print("\n💡 ۱۰ کلمه با بیشترین شباهت معنایی به زمینه‌ی اطراف «{}»:".format(keyword))
for word, score in semantic_top:
    print(f"{word}: {score:.4f}")
